In [65]:
#import findspark
#findspark.init('/home/max/Downloads/spark-2.4.0-bin-hadoop2.7')
# from pyspark.sql import SparkSession
# from pyspark.sql import Row



In [66]:
#spark= SparkSession.builder.getOrCreate()

In [67]:
from io import StringIO
import matplotlib.pyplot as plt
import random
import numpy as np
import pandas as pd
import datetime
from datetime import timedelta
import csv
import sys
from pympler.asizeof import asizeof
import boto3
import time

import s3fs
from fastparquet import write

import pyarrow.parquet as pq

In [68]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_colwidth', -1)

In [69]:
# import code counts, and convert to dict with probabilities
code_counts = pd.read_csv('code_counts.csv')
code_counts.columns = ['code', 'count']
code_counts = code_counts[code_counts['count'] > 100]
code_counts['prop'] = code_counts['count']/code_counts['count'].sum()
#code_counts.head()
code_counts = dict(zip(code_counts['code'], code_counts['prop']))

In [70]:
unique_codes = list(code_counts.keys())
#print(list(unique_codes))

In [71]:
# get random code based on the weight (probability) of each code
def get_random_code(dct):
    rand_val = random.random()
    total = 0
    for k, v in dct.items():
        total += v
        if rand_val <= total:
            return k
    assert False, 'unreachable'
    
get_random_code(code_counts)

'H04'

In [72]:
with open('filtered_text.txt') as f:
    terms = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
terms = [x.strip() for x in terms] 
random.shuffle(terms)

In [84]:
#start_value = 1
start_time_shift =20
random_walk_weight = 0.03
random_walk_variance = 1
seasonality_1_freq = 50 #lower is shorter
seasonality_1_weight = 5
seasonality_2_freq = 7 #lower is shorter
seasonality_2_weight = 2

num_words_in_text = 100 # number of words in each patent text

sample_size = 1000

trend = 0.02
#exponent = 1
time_reg = 10000
exponent = 0.553
n_samples = 10
bucket_name = 'patents-bucket'

times = [0,0,0,0,0,0]
dfs = []

day = 0

# store dataframe to S3 in CSV format
def store_data(df, file_num, csv=True):
    if csv:
        csv_buffer = StringIO()
        df.to_csv(csv_buffer, index=False)
        #df.to_parquet(buffer)
        s3_resource = boto3.resource('s3')
        s3_resource.Object(bucket_name, 'patents-csv/patents_'+str(file_num)+'.csv').put(Body=csv_buffer.getvalue())
        #s3_resource.Object(bucket_name, 'patents-parquet/patents_'+str(file_num)+'.parquet').put(Body=buffer.getvalue())
    
    else:
        s3_url = 's3://patents-bucket/patents-test/patents2_'+str(file_num)+'.csv'
        df.to_parquet(s3_url)
    
    dfs.append(df)
    
# Get number of samples to be made for one day
def get_num_samples(day):
    #Assume no start at y = 0
    random_walk = np.random.normal(0,random_walk_variance)
    time = day + start_time_shift

    y = (np.sin(time/seasonality_1_freq))*seasonality_1_weight #Add seasonality 1
    y = y + np.sin(time/seasonality_2_freq)*seasonality_2_weight #Add seasonality 2
    #y = y + (trend * (time ** exponent)) #Add trend
    y = y + (trend*(time ** (exponent * (1 + time / time_reg) ) ) )

    # Random walk component
    random_walk += np.random.normal(0,random_walk_variance)
    
    return max(0, y + random_walk * random_walk_weight)


def create_datasets(filesize, total_days, day):
    #print("Storing", filesize/1000000, "MB in each file")
    start_time = time.time()
    rows = []
    file_num = 0

    while day < total_days:
        #if i%10000 == 0:
            #print(i, sys.getsizeof(rows))
        
        #s1 = time.time()    
        if sys.getsizeof(rows) > filesize: # larger than filesize (120mb?), store file
            df = pd.DataFrame(rows, columns=['date', 'code', 'text'])
            #print("stored", sys.getsizeof(rows)/1000000, "MB in file num", file_num)
            store_data(df, file_num)
            print("stored file number", file_num, ", current day:",day,day/total_days,"%), time:", time.time()-start_time)
            rows = []
            file_num+=1
        #e1 = time.time()
        #times[3] += e1-s1
            
        #s2 = time.time()  
        num_samples = get_num_samples(day)
        #e2 = time.time()
        #times[4] += e2-s2
        
        #s3 = time.time()
        for n in range(int(round(num_samples))): #Create n entries
            #s = time.time()
            date = datetime.datetime(1968, 1, 1, 0, 0) + timedelta(days=int(day))
            current_sample = day+n
            #e = time.time()
            #times[0] += e-s
            #sample = terms[current_sample:current_sample+100]
            #text = " ".join(random.sample(terms, num_words_in_text)) # TOO SLOW
            #s = time.time()
            rnd = random.randint(1,len(terms)-101)
            text = " ".join(terms[rnd:rnd+100])
            #e = time.time()
            #times[1] += e-s
            #s = time.time()
            #rows.append([date, get_random_code(code_counts), text])
            
            rows.append([date, random.choice(unique_codes), text])
            #e = time.time()
            #times[2] += e-s
        #e3 = time.time()
        #times[2] += e3-s3
        
        day+=1
    end_time = time.time()
    print("Total time passed:", end_time-start_time)
        
csv_filesize = 2000000
parquet_filesize = 2000000

max_total_size = 3000000 #3mb
#max_files = int(max_total_size/filesize)
total_days = 18250
start_day = 1
create_datasets(csv_filesize, total_days, start_day)


stored file number 0 , current day: 7920 0.43397260273972604 %), time: 31.468837022781372
stored file number 1 , current day: 9027 0.49463013698630137 %), time: 84.36957097053528
stored file number 2 , current day: 9677 0.5302465753424658 %), time: 105.04168319702148
stored file number 3 , current day: 10139 0.5555616438356165 %), time: 151.0017602443695
stored file number 4 , current day: 10498 0.5752328767123288 %), time: 172.64944505691528
stored file number 5 , current day: 10792 0.5913424657534246 %), time: 202.9931571483612
stored file number 6 , current day: 11040 0.6049315068493151 %), time: 255.95258808135986
stored file number 7 , current day: 11255 0.6167123287671233 %), time: 296.00857520103455
stored file number 8 , current day: 11444 0.6270684931506849 %), time: 319.4930810928345
stored file number 9 , current day: 11614 0.6363835616438356 %), time: 359.38517022132874
stored file number 10 , current day: 11767 0.6447671232876713 %), time: 436.86851596832275
stored file nu

KeyboardInterrupt: 

In [64]:
datetime.datetime(1968, 1, 1, 0, 0) + timedelta(days=int(18250))

datetime.datetime(2017, 12, 19, 0, 0)

In [35]:
           
s = time.time()
for i in range(10000000):
    rnd = random.randint(1,len(terms)-101)
    terms[rnd:rnd+100]
e = time.time()
print( e-s)

27.536070108413696
